# UAVSAR Wetlands
## 1. Get UAVSAR Data
## 2. Convert to Geotiff & Subset with GDAL

Bonanza Creek 2017: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=bonanE_03604_17100_005_170919_L090_CX_01#data

Bonanza Creek 2018: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=bonanE_03604_18051_006_180827_L090_CX_01#data

Bonanza Creek 2019: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=bonanE_03604_19063_010_190913_L090_CX_01#data

Bonanza Creek 2019: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=bonanE_03604_19064_004_190914_L090_CX_01#data


---------------------------------

Southern California Coast: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=SSurge_15305_14170_007_141120_L090_CX_02#data

Redwood Forest: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=RedWod_00301_09088_000_091104_L090_CX_01#data

Louisiana Marsh: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=LAmrsh_27906_10054_008_100624_L090_CX_02#data

Costa Rica Mangroves: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=corcov_36026_10019_007_100210_L090_CX_01#data


In [ ]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y

In [ ]:
import gdal

Bonanza Creek Extent [﻿-148.1640, -147.9379, 64.66, 64.7707]

# GDAL Calls

1. Download HHHH GRD File 
2. Make .hdr file by running Edit_BuildUAVSARhdr.py(need to change samples, lines, and left corner lat/lon)

python /Users/peacock/Desktop/Edit_BuildUAVSARhdr.py -i corcov_36026_10019_007_100210_L090_CX_01.ann.txt -r corcov_36026_10019_007_100210_L090HHHH_CX_01.grd -p HHHH

python /Users/peacock/Desktop/Edit_BuildUAVSARhdr.py -i LAmrsh_27906_10054_008_100624_L090_CX_02.ann.txt -r LAmrsh_27906_10054_008_100624_L090HHHH_CX_02.grd -p HHHH

3. Run through Naiara's script 'raster_read.py' to generate a GeoTIFF from the .hdr and .grd files
4. This GeoTIFF and the Pauli GeoTIFFs have a black outline in areas of no data. Need to get rid of this:
5. In coherence file, set 0 as Nodata

gdal_translate -of GTiff -a_nodata 0 -a_srs EPSG:4326 SSurge_15305_14170_007_141120_L090HHHH_CX_02.grd.tif SSurge_15305_14170_007_141120_L090HHHH_CX_02_nodata.grd.tif

gdal_translate -of GTiff -a_nodata 0 -a_srs EPSG:4326 corcov_36026_10019_007_100210_L090HHHH_CX_01.grd.tif corcov_36026_10019_007_100210_L090HHHH_CX_01_nodata.grd.tif

gdal_translate -of GTiff -a_nodata 0 -a_srs EPSG:4326 LAmrsh_27906_10054_008_100624_L090HHHH_CX_02.grd.tif LAmrsh_27906_10054_008_100624_L090HHHH_CX_02_nodata.grd.tif

6. Now set nodata to -999 and only grab values above this

gdal_calc.py -A SSurge_15305_14170_007_141120_L090HHHH_CX_02_nodata.grd.tiff --outfile ssurge_out.tif --NoDataValue -999 --calc="1*(A>-999)”

gdal_calc.py -A corcov_36026_10019_007_100210_L090HHHH_CX_01_nodata.grd.tif --outfile corcov_out.tif --NoDataValue -999 --calc="1*(A>-999)”

gdal_calc.py -A LAmrsh_27906_10054_008_100624_L090HHHH_CX_02_nodata.grd.tif --outfile lamarsh_out.tif --NoDataValue -999 --calc="1*(A>-999)”

7. Create polygon of this new subset

gdal_polygonize.py -8 -b 1 -f "ESRI Shapefile" ssurge_out.tif ssurge_out

gdal_polygonize.py -8 -b 1 -f "ESRI Shapefile" corcov_out.tif corcov_out

gdal_polygonize.py -8 -b 1 -f "ESRI Shapefile" lamarsh_out.tif lamarsh_out

8. Crop Pauli tifs to this extent. 

gdalwarp -of GTiff -cutline ssurge_out/out.shp -crop_to_cutline -dstalpha SSurge_15305_16098_003_161114_L090_CX_01_pauli.tif SSurge_15305_16098_003_161114_L090_CX_01_pauli_cropped.tif

gdalwarp -of GTiff -cutline corcov_out/out.shp -crop_to_cutline -dstalpha corcov_36026_10019_007_100210_L090_CX_01_pauli.tif corcov_36026_10019_007_100210_L090_CX_01_pauli_cropped.tif

gdalwarp -of GTiff -cutline lamarsh_out/out.shp -crop_to_cutline -dstalpha LAmrsh_27906_10054_008_100624_L090_CX_02_pauli.tif LAmrsh_27906_10054_008_100624_L090_CX_02_pauli_cropped.tif


######### Make Map Tiles #############
python
Import gdal2tiles

gdal2tiles.generate_tiles("corcov_36026_10019_007_100210_L090_CX_01_pauli.tif", "./corcov10", zoom='5-14')
gdal2tiles.generate_tiles("RedWod_00301_09088_000_091104_L090_CX_01_pauli.tif", "./redwod09", zoom='5-14')
gdal2tiles.generate_tiles("LAmrsh_27906_10054_008_100624_L090_CX_02_pauli.tif", "./lamarsh10", zoom='5-14')
gdal2tiles.generate_tiles("SSurge_15305_16098_003_161114_L090_CX_01_pauli_cropped.tif", "./ssurge16", zoom='5-14')



gdal2tiles.generate_tiles("SanAnd_26530_17122_002_171102_L090_CX_01_pauli.tif", "./thomasnovember", zoom='5-14')
gdal2tiles.generate_tiles("SanAnd_26530_18001_007_180205_L090_CX_01_pauli", "./thomasfeb", zoom='5-14')